# Control Script

In [1]:
import azureml.core
from azureml.core import Workspace

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Failure while loading azureml_run_type_providers. Failed to load entrypoint hyperdrive = azureml.train.hyperdrive:HyperDriveRun._from_run_dto with exception (pyOpenSSL 20.0.1 (c:\users\awnya\miniconda3\envs\ml\lib\site-packages), Requirement.parse('pyopenssl<20.0.0'), {'azureml-core'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (pyOpenSSL 20.0.1 (c:\users\awnya\miniconda3\envs\ml\lib\site-packages), Requirement.parse('pyopenssl<20.0.0'), {'azureml-core'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.PipelineRun = azureml.pipeline.core.run:PipelineRun._from_dto with exception (pyOpenSSL 20.0.1 (c:\users\awnya\miniconda3\envs\ml\lib\site-packages), Requirement.parse('pyopenssl<20.0.0'), {'azureml-core'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.ReusedStepRun = azureml.pipeline.core.run:StepRun._from_

In [2]:
# load workspace configuration from the config.json file in the current folder.
ws = Workspace.from_config()
print(ws.name, ws.location, ws.resource_group, sep='\t')

Performing interactive authentication. Please follow the instructions on the terminal.
Note, we have launched a browser for you to login. For old experience with device code, use "az login --use-device-code"
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
3DCV	eastus2	3DCV-resource-group


In [3]:
experiment_name = 'rgb_model0'

from azureml.core import Experiment
exp = Experiment(workspace=ws, name=experiment_name)

In [4]:
# specify compute target
compute_name = 'gpu-compute-3dcv'
compute_target = ws.compute_targets[compute_name]


## UPLOAD DATA

In [5]:
# upload dataset to azure
datastore = ws.get_default_datastore()
src_dir = '../../RGB_dataset'
datastore.upload(src_dir=src_dir,
                 target_path='datasets/rgb_dataset',
                 overwrite=True)


_cup\food_cup_1_4_104.png
Uploaded ../../RGB_dataset\train\food_cup\food_cup_1_4_104.png, 99222 files out of an estimated total of 209151
Uploading ../../RGB_dataset\train\food_cup\food_cup_1_4_105.png
Uploaded ../../RGB_dataset\train\food_cup\food_cup_1_4_105.png, 99223 files out of an estimated total of 209151
Uploading ../../RGB_dataset\train\food_cup\food_cup_1_4_106.png
Uploaded ../../RGB_dataset\train\food_cup\food_cup_1_4_106.png, 99224 files out of an estimated total of 209151
Uploading ../../RGB_dataset\train\food_cup\food_cup_1_4_107.png
Uploaded ../../RGB_dataset\train\food_cup\food_cup_1_4_107.png, 99225 files out of an estimated total of 209151
Uploading ../../RGB_dataset\train\food_cup\food_cup_1_4_108.png
Uploaded ../../RGB_dataset\train\food_cup\food_cup_1_4_108.png, 99226 files out of an estimated total of 209151
Uploading ../../RGB_dataset\train\food_cup\food_cup_1_4_109.png
Uploaded ../../RGB_dataset\train\food_cup\food_cup_1_4_109.png, 99227 files out of an estimate

KeyboardInterrupt: 

In [10]:
# upload embedding models
src_dir = 'C:/Users/awnya/Documents/Projects/RGBD Object Classification/3d-object-classification/models'
datastore.upload(src_dir=src_dir,
                 target_path='embedding_models',
                 overwrite=True)

Uploading an estimated of 4 files
Uploading C:/Users/awnya/Documents/Projects/RGBD Object Classification/3d-object-classification/models\bit_m-r50x1_1\variables\variables.index
Uploaded C:/Users/awnya/Documents/Projects/RGBD Object Classification/3d-object-classification/models\bit_m-r50x1_1\variables\variables.index, 1 files out of an estimated total of 4
Uploading C:/Users/awnya/Documents/Projects/RGBD Object Classification/3d-object-classification/models\bit_m-r50x1_1\saved_model.pb
Uploaded C:/Users/awnya/Documents/Projects/RGBD Object Classification/3d-object-classification/models\bit_m-r50x1_1\saved_model.pb, 2 files out of an estimated total of 4
Uploading C:/Users/awnya/Documents/Projects/RGBD Object Classification/3d-object-classification/models\bit_m-r50x1_1\variables\variables.data-00000-of-00001
Uploaded C:/Users/awnya/Documents/Projects/RGBD Object Classification/3d-object-classification/models\bit_m-r50x1_1\variables\variables.data-00000-of-00001, 3 files out of an estima

$AZUREML_DATAREFERENCE_cc8251a3501f4da9855861e9b58ce8d3

## RUN TRAINING SCRIPT

In [1]:
from azureml.core import Workspace
from azureml.core import Experiment
from azureml.core import Environment
from azureml.core import ScriptRunConfig
from azureml.core import Dataset

Failure while loading azureml_run_type_providers. Failed to load entrypoint hyperdrive = azureml.train.hyperdrive:HyperDriveRun._from_run_dto with exception (pyOpenSSL 20.0.1 (c:\users\awnya\miniconda3\envs\ml\lib\site-packages), Requirement.parse('pyopenssl<20.0.0'), {'azureml-core'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (pyOpenSSL 20.0.1 (c:\users\awnya\miniconda3\envs\ml\lib\site-packages), Requirement.parse('pyopenssl<20.0.0'), {'azureml-core'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.PipelineRun = azureml.pipeline.core.run:PipelineRun._from_dto with exception (pyOpenSSL 20.0.1 (c:\users\awnya\miniconda3\envs\ml\lib\site-packages), Requirement.parse('pyopenssl<20.0.0'), {'azureml-core'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.ReusedStepRun = azureml.pipeline.core.run:StepRun._from_

In [2]:
ws = Workspace.from_config()
datastore = ws.get_default_datastore()
dataset = Dataset.File.from_files(path=(datastore, 'datasets/rgb_dataset'))
# embedding_model = Dataset.File.from_files(path=(datastore, 'embedding_models/bit_m-r50x1_1'))

experiment_name = 'rgb_model0'
experiment = Experiment(workspace=ws, name=experiment_name)

In [3]:
compute_name = 'compute-cpu'

config = ScriptRunConfig(
    source_directory='./src',
    script='train.py',
    compute_target=compute_name,
    arguments=[
        '--data_path', dataset.as_named_input('input').as_mount(),
        ],
)

In [4]:
# set up 3dcv environment
# env_path = '.azureml/3dcv-env.yml'
# env = Environment.from_conda_specification(
#     name='3dcv',
#     file_path=env_path
# )
# env.register(workspace=ws);
# config.run_config.environment = env

config.run_config.environment = ws.environments['3dcv']

In [5]:
run = experiment.submit(config)
aml_url = run.get_portal_url()
print("Submitted to compute cluster. Click link below")
print("")
print(aml_url)

Submitted to compute cluster. Click link below

https://ml.azure.com/experiments/rgb_model0/runs/rgb_model0_1609058877_1422d835?wsid=/subscriptions/daade51c-81b8-4483-af67-14e3ec399e5d/resourcegroups/3DCV-resource-group/workspaces/3DCV


In [6]:
run

Experiment,Id,Type,Status,Details Page,Docs Page
rgb_model0,rgb_model0_1609058877_1422d835,azureml.scriptrun,Starting,Link to Azure Machine Learning studio,Link to Documentation


In [7]:
# specify show_output to True for a verbose log
run.wait_for_completion (show_output=True) 

RunId: rgb_model0_1609058877_1422d835
Web View: https://ml.azure.com/experiments/rgb_model0/runs/rgb_model0_1609058877_1422d835?wsid=/subscriptions/daade51c-81b8-4483-af67-14e3ec399e5d/resourcegroups/3DCV-resource-group/workspaces/3DCV

Streaming azureml-logs/55_azureml-execution-tvmps_1e98353a64ef60bf946c5e8b63ebf469464664ba17efa2585d6239daa176405a_d.txt

2020-12-27T08:48:07Z Starting output-watcher...
2020-12-27T08:48:07Z IsDedicatedCompute == True, won't poll for Low Pri Preemption
2020-12-27T08:48:08Z Executing 'Copy ACR Details file' on 10.0.0.5
2020-12-27T08:48:08Z Copy ACR Details file succeeded on 10.0.0.5. Output: 
>>>   
>>>   
Login Succeeded
Using default tag: latest
latest: Pulling from azureml/azureml_0f5926f4b7d035d32b6243bc3f3b5841
Digest: sha256:78a32f2ddd340c86ad48aba174f925495c274d8612cefb168874e4ab9516725b
Status: Image is up to date for viennaglobal.azurecr.io/azureml/azureml_0f5926f4b7d035d32b6243bc3f3b5841:latest
viennaglobal.azurecr.io/azureml/azureml_0f5926f4b7

{'runId': 'rgb_model0_1609058877_1422d835',
 'target': 'compute-cpu',
 'status': 'Completed',
 'startTimeUtc': '2020-12-27T08:48:07.195699Z',
 'endTimeUtc': '2020-12-27T08:53:53.409876Z',
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': 'cf01f0e5-625d-483a-a6bf-ac8a877910a1',
  'azureml.git.repository_uri': 'https://github.com/Awni00/3d-object-classification.git',
  'mlflow.source.git.repoURL': 'https://github.com/Awni00/3d-object-classification.git',
  'azureml.git.branch': 'main',
  'mlflow.source.git.branch': 'main',
  'azureml.git.commit': 'a7b429123e2014593b2817708a66707a9295fcdb',
  'mlflow.source.git.commit': 'a7b429123e2014593b2817708a66707a9295fcdb',
  'azureml.git.dirty': 'True',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [{'dataset': {'id': '53f8c08a-6f9a-4893-b74d-f1a4f37d6951'}, 'consumptionDetails': {'type': 'RunInput', 'inputName': 'input', 'mechani

In [8]:
run.get_file_names()

['azureml-logs/55_azureml-execution-tvmps_1e98353a64ef60bf946c5e8b63ebf469464664ba17efa2585d6239daa176405a_d.txt',
 'azureml-logs/65_job_prep-tvmps_1e98353a64ef60bf946c5e8b63ebf469464664ba17efa2585d6239daa176405a_d.txt',
 'azureml-logs/70_driver_log.txt',
 'azureml-logs/75_job_post-tvmps_1e98353a64ef60bf946c5e8b63ebf469464664ba17efa2585d6239daa176405a_d.txt',
 'azureml-logs/process_info.json',
 'azureml-logs/process_status.json',
 'logs/azureml/79_azureml.log',
 'logs/azureml/dataprep/backgroundProcess.log',
 'logs/azureml/dataprep/backgroundProcess_Telemetry.log',
 'logs/azureml/dataprep/engine_spans_99e43322-fa0d-4fde-9eb8-50e4b67764ad.jsonl',
 'logs/azureml/dataprep/engine_spans_c75e5de5-3dd6-437c-b926-dd30618492e3.jsonl',
 'logs/azureml/dataprep/python_span_3e0327c3-c8a9-4980-a2e8-1dd609bf432d.jsonl',
 'logs/azureml/dataprep/python_span_99e43322-fa0d-4fde-9eb8-50e4b67764ad.jsonl',
 'logs/azureml/dataprep/python_span_c75e5de5-3dd6-437c-b926-dd30618492e3.jsonl',
 'logs/azureml/job_pr

In [9]:
# register model 
model = run.register_model(model_name='rgb_model0', model_path='outputs/rgb_model0.h5')
print(model.name, model.id, model.version, sep='\t')

rgb_model0	rgb_model0:1	1
